In [2]:
### test image data for train ###
# 230904
# list2batch

In [1]:
import sys
sys.path.append('/home/gpuadmin/dev/Trajectory_Prediction/traffino')
# sys.path.append('C:/Users/NGN/dev/Traffino/TRAFFINO')

import argparse
import gc
import logging
import os
import sys
import time

from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim

# from data.loader_image import img_loader
from data.loader_traffic import data_loader
import torchvision.transforms as T

In [2]:
from losses import gan_g_loss, gan_d_loss, l2_loss
from losses import displacement_error, final_displacement_error

In [3]:
from traffino.models_traffic_2 import TrajectoryGenerator, TrajectoryDiscriminator, SemanticImageEncoder # traffic model 불러오기

from traffino.utils import int_tuple, bool_flag, get_total_norm
from traffino.utils import relative_to_abs, get_dset_path

In [4]:
torch.backends.cudnn.benchmark = True # cudnn의 benchmark를 통해 최적 backend 연산을 찾는 flag를 true로 하겠음

In [5]:
parser = argparse.ArgumentParser()
FORMAT = '[%(levelname)s: %(filename)s: %(lineno)4d]: %(message)s'
logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
logger = logging.getLogger(__name__)

In [6]:
from typing import Tuple
class CreateArg():
    def __init__(self):
        # Dataset options
        self.dataset_name = 'waterloo'
        self.delim = '\t'
        self.loader_num_workers = 4
        self.obs_len = 8
        self.pred_len = 8
        self.skip = 1
        # Optimization
        self.batch_size = 4                 #### batch_size test !!!
        self.num_iterations = 1000 # 10000
        self.num_epochs = 50 # 200                     
        # Model Options
        self.embedding_dim = 64
        self.num_layers = 1
        self.dropout = 0.0
        self.batch_norm = 0 
        self.mlp_dim = 1024
        # self.default_backbone= 'resnet18'
                                    
        # Generator Options
        self.encoder_h_dim_g = 64
        self.decoder_h_dim_g = 128
        self.noise_dim : Tuple[int] = (0, 0) # default=None # type=int_tuple
        self.noise_type = 'gaussian'
        self.noise_mix_type = 'ped'
        self.clipping_threshold_g = 0 # type=float
        self.g_learning_rate = 5e-4 # type=float 
        self.g_steps = 1

        # Pooling Options
        self.pooling_type = 'pool_net' 
        self.pool_every_timestep = 1 # type=bool_flag

        # Pool Net Option
        self.bottleneck_dim = 1024 # type=int

        # Social Pooling Options
        self.neighborhood_size = 1024 # type=float
        self.grid_size = 8 # type=int

        # Discriminator Options
        self.d_type = 'local' # type=str
        self.encoder_h_dim_d = 64 # type=int
        self.d_learning_rate = 5e-4 # type=float
        self.d_steps = 2 # type=int        
        self.clipping_threshold_d = 0 # type=float  

        # Loss Options
        self.l2_loss_weight = 0 # type=float 
        self.best_k = 1 # type=int 

        # Output
        self.output_dir = os.getcwd()+ '/output2'
        self.print_every = 5 # type=int
        self.checkpoint_every = 100 # type=int
        self.checkpoint_name = 'checkpoint_traffic' 
        self.checkpoint_start_from = None
        self.restore_from_checkpoint = 1 # type=int
        self.num_samples_check = 5000 # type=int        

        # Misc
        self.use_gpu = 1 # type=int
        self.timing = 0 # type=int
        self.gpu_num = "1" # type=str   


args = CreateArg() 

In [7]:
print(args.batch_size)
print(args.num_iterations)
print(args.checkpoint_name)
print(args.output_dir)
print(args.restore_from_checkpoint )

4
1000
checkpoint_traffic
/home/gpuadmin/dev/Trajectory_Prediction/traffino/output2
1


In [8]:
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight)


def get_dtypes(args):
    long_dtype = torch.LongTensor
    float_dtype = torch.FloatTensor
    if args.use_gpu == 1:
    # use_gpu == 1:
        long_dtype = torch.cuda.LongTensor
        float_dtype = torch.cuda.FloatTensor
    return long_dtype, float_dtype

In [9]:
long_dtype, float_dtype = get_dtypes(args)

In [10]:
# train_path = "C:/Users/NGN/dev/Traffino/TRAFFINO/traffino/datasets/img/train/"
train_path = "/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/train/"
# val_path = get_dset_path(args.dataset_name, 'val')
# val_path = 'C:/Users/NGN/dev/Traffino/TRAFFINO/traffino/datasets/img/val/'
val_path = "/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/val/"

In [11]:
transforms = T.Compose([T.ToTensor()]) # Normalize the pixels and convert to tensor.

logger.info("Initializing train dataset")

train_dset, train_loader = data_loader(args, train_path) # train_dset은 TrajectoryDataset, train_loader는 DataLoader


# full_dataset = ImageDataset(
#     "C:/Users/NGN/dev/Traffino/TRAFFINO/traffino/datasets/waterloo/img_test/769/", 
#     transforms
#     )

# logger.info("Initializing val dataset")
# val_det = data_loader(args, val_path)

[INFO: 2429258518.py:    3]: Initializing train dataset
['0771_prep.txt', '0776_prep.txt', '0775_prep.txt', '0778_prep.txt', '0777_prep.txt', '0770_prep.txt', '0769_prep.txt', '0779_prep.txt']
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/train/0769_prep.txt
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/train2/0769_prep2.txt
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/train2/0769_prep3.txt
['769', '770', '771', '775', '776', '777', '778', '779']
0
img_path:/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/train/0770_prep.txt
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/train2/0770_prep2.txt
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/train2/0770_prep3.txt
['769', '770', '771', '775', '776', '777', '778', '779']
1
img_path:/home/gpuadmin/dev/Trajectory_Prediction/traffino/dat

In [12]:
print(args.batch_size)

4


In [15]:
print(train_dset)
print(train_loader)
# print(val_det)

In [16]:
print(len(train_dset)) # 이미지 개수
print(len(train_loader))  # 이미지 개수 / batch_size
# print(len(val_det))

5385
1347


In [17]:
# train_dset에 대한 output 출력
print(train_dset.fet_map)



{'/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769': tensor([[[[0.0000e+00, 6.8419e-01, 0.0000e+00,  ..., 6.9899e-03,
           0.0000e+00, 9.5197e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [2.1932e+00, 2.1953e+00, 0.0000e+00,  ..., 0.0000e+00,
           1.0288e-01, 0.0000e+00],
          ...,
          [2.4005e+00, 0.0000e+00, 2.7180e-01,  ..., 0.0000e+00,
           0.0000e+00, 1.0479e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 7.8761e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [1.0013e-01, 0.0000e+00, 3.6137e+00,  ..., 0.0000e+00,
           2.9627e-01, 6.0187e-01],
          [0.0000e+00, 1.6036e+00, 5.5664e+00,  ..., 0.0000e+00,
           8.9287e-01, 1

In [20]:
#### --- --- ####

In [19]:
print(train_loader.img_list)


AttributeError: 'DataLoader' object has no attribute 'img_list'

In [19]:
print(train_dset.tensor_list)


[tensor([[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
         [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
         [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
         ...,
         [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
         [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0118, 0.0078, 0

In [20]:
print(type(train_dset.tensor_list))
print(len(train_dset.tensor_list))

<class 'list'>
727


In [21]:
print(type(train_dset.tensor_list[0])) # 0번째 이미지->tensor로
print(train_dset.tensor_list[0])
print(len(train_dset.tensor_list[0])) # 차원의 수
print(train_dset.tensor_list[0].shape) # 이미지 shape

<class 'torch.Tensor'>
tensor([[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
         [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
         [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
         ...,
         [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
         [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  

In [38]:
print(type(train_dset.tensor_list)) # 
print(train_dset.tensor_list)
print(len(train_dset.tensor_list)) # 프레임의 수


<class 'list'>
[tensor([[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
         [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
         [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
         ...,
         [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
         [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.

In [24]:
# before concatenation
# print(type(train_dset.list_ten_img[0]))
# print(train_dset.list_ten_img[0].shape)

<class 'torch.Tensor'>
torch.Size([3, 360, 640])


In [22]:
print(type(train_dset.list_ten_img))
print(train_dset.list_ten_img.shape)
# 3*727

AttributeError: 'ImageDataset' object has no attribute 'list_ten_img'

In [27]:
def list2batch(seq):
    # assemble a list of elements to batch, batch_idx: 0th dimension
    stacked = torch.tensor(seq[0]).unsqueeze(0)
    i = 1
    l = len(seq) # 리스트 길이 (이미지 파일 수)
    while i < l:
        stacked = torch.cat((stacked, torch.tensor(seq[i]).unsqueeze(0)), 0)
        i += 1
    return stacked

In [30]:
batch_img_list=list2batch(train_dset.tensor_list)
print(batch_img_list)
print(batch_img_list.shape)
print(type(batch_img_list))

/home/gpuadmin/anaconda3/envs/traffino/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gpuadmin/anaconda3/envs/traffino/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


tensor([[[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
          [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
          [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
          [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

         [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
          [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
          [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
          ...,
          [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
          [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
          [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0000,  ..., 0

In [34]:
print(len(batch_img_list))
print(batch_img_list[0].shape)

print(len(batch_img_list))
print(batch_img_list.shape) # (frame 수, channel, h, w)

727
torch.Size([3, 360, 640])
727
torch.Size([727, 3, 360, 640])
